## Demo

In [1]:
import pandas as pd
from core.retriever import Retriever

r = Retriever(
    csv_path="data/data.csv",
    model_name="Qwen/Qwen3-Embedding-0.6B",  retrieve_k=64,
    reranker_name="BAAI/bge-reranker-v2-m3", top_k=8,  # optional
    rebuild_index=False
)

r.load()

query = "hicup"
nodes = r.search(query)

pd.DataFrame(
    [{"id": n.metadata["id"], "text": n.text, "score": n.score} for n in nodes]
).head(3)

,id,text,score
0,7ec19a40-fe3a-4f1d-9167-99f26b76b8d3,วัคซีนฮิบคือวัคซีนที่ป้องกันการติดเชื้อแบคทีเร...,0.320537
1,b712ed08-b1e4-4b5d-80cf-cfb20877e791,การรักษาศัลยกรรมแตกหักฮุคของกระดูกฮูเมตมีสองวิ...,0.315557
2,2c5f0cc2-5278-41ac-a70e-4a10e907625c,Hypocitraturia คือภาวะที่มีการขับ citrate ในน้...,0.314804
